In [1]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array, array_to_img

In [2]:
from tensorflow.keras import Input
from tensorflow.keras import layers, Model

input_img = Input(shape=(600, 600, 3))

print(input_img)
x = layers.Conv2D(128, kernel_size = 3, strides = 1,  activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
print(x)
x = layers.Conv2D(64, kernel_size = 3, strides = 1, activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
print(x)
x = layers.Conv2D(32, kernel_size = 3, strides = 1, activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
print(encoded)

x = layers.Conv2DTranspose(64, (3, 3), strides = 2, activation='relu', padding='same')(encoded)
print(x)
x = layers.Conv2DTranspose(128, (3, 3), strides = 2, activation='relu', padding='same')(x)
print(x)
decoded = layers.Conv2DTranspose(3, (3, 3), strides = 2, activation='relu', padding='same')(x)
print(decoded)
autoencoder = Model(input_img,decoded)

Tensor("input_1:0", shape=(None, 600, 600, 3), dtype=float32)
Tensor("max_pooling2d/Identity:0", shape=(None, 300, 300, 128), dtype=float32)
Tensor("max_pooling2d_1/Identity:0", shape=(None, 150, 150, 64), dtype=float32)
Tensor("max_pooling2d_2/Identity:0", shape=(None, 75, 75, 32), dtype=float32)
Tensor("conv2d_transpose/Identity:0", shape=(None, 150, 150, 64), dtype=float32)
Tensor("conv2d_transpose_1/Identity:0", shape=(None, 300, 300, 128), dtype=float32)
Tensor("conv2d_transpose_2/Identity:0", shape=(None, 600, 600, 3), dtype=float32)


In [3]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [4]:
#Loading data
import os
trainfalsedirpath = 'Folds/3folds/fold1/Train/False'
trainfalsedir = os.listdir(trainfalsedirpath)


xtrain = np.zeros([6,600,600,3])
xval = np.zeros([6,600,600,3])

for k,v in enumerate(trainfalsedir[:5]):
    xtrain[k] = img_to_array(load_img(trainfalsedirpath + '/' + v))

for k,v in enumerate(trainfalsedir[5:10]):
    xval[k] = img_to_array(load_img(trainfalsedirpath + '/' + v))


In [5]:
autoencoder.fit(
    xtrain,
    xtrain,
    epochs = 1,
    batch_size = 2,
    shuffle = True,
    validation_data = (xval, xval)
)

3/3 [==============================] - 12s 4s/step - loss: 404.6444 - val_loss: -651.3224


In [20]:
a = autoencoder.predict(np.expand_dims(img_to_array(load_img(trainfalsedirpath + '/' + trainfalsedir[21])), axis = 0))